#  Установка библиотек

In [1]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:39
🔁 Restarting kernel...


In [2]:
!mamba install -c conda-forge rdkit chembl_structure_pipeline



                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.8.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit', 'chembl_structure_pipeline']

conda-forge/linux-64     Using cache
conda-forge/noarch     

# Импорт библиотек

In [3]:
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors
from matplotlib import pyplot as plt
%matplotlib inline
from typing import Dict, List, Optional
from pathlib import Path

# Загрузка данных

In [4]:
!wget https://www.dropbox.com/s/5b05tivi01a43np/delaney-processed.csv
!wget https://www.dropbox.com/s/dsrl00tj4zjmqbz/Lipophilicity.csv

--2021-05-09 10:15:12--  https://www.dropbox.com/s/5b05tivi01a43np/delaney-processed.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5b05tivi01a43np/delaney-processed.csv [following]
--2021-05-09 10:15:12--  https://www.dropbox.com/s/raw/5b05tivi01a43np/delaney-processed.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc193d6ec575f5aed1294ed33b55.dl.dropboxusercontent.com/cd/0/inline/BOKikRCrpXXzv5NkTtzSR7-X6xwCiKsbNg-cHIytGll6iso56Mnt5Y-1-eWoB-LDq77rKYu2mKYJB4dD7Cwtq8aNX9QyHELEHmHZ1paWkodo--aWY0Fw1gSdfKYabh2uxxTWRWAu8DdnL3jGT5gwgV7O/file# [following]
--2021-05-09 10:15:12--  https://uc193d6ec575f5aed1294ed33b55.dl.dropboxusercontent.com/cd/0/inline/BOKikRCrpXXzv5NkTtzSR7-X6xwCiKsbNg-cHIytGll6iso56Mnt5Y-1-eWoB-L

# Предобработка данных

1. Сделайте стандартизацию SMILES в датасете (используйте код с TautomerCanonicalizer)

2. Проверьте, появились ли дубликаты в обработанных SMILES. Удалите их при помощи метода drop_duplicates()



In [8]:
class DatasetsHolder:
    @staticmethod
    def read_datasets(inp_folder_path):
        # return pandas DataFrame
        df = pd.read_csv(inp_folder_path)
        return df


class StandardizeDatasets:
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        "crete typical standartization of one smiles"
        mol = Chem.MolFromSmiles(smi)
        smi1 = Chem.MolToSmiles(mol)
        return smi1

    def standardize(self, inp_path: Path, out_path: Path):
        "apply standartization to all smiles"
        dt = DatasetsHolder()
        df = dt.read_datasets(inp_path)
        sd = StandardizeDatasets()
        df['standardized_smiles'] = df['smiles'].apply(sd.standardize_smiles)
        df.to_csv(out_path)


class StandardizeTautomers(StandardizeDatasets):
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        "apply TautomerCanonicalizer() to standartization"
        tc = TautomerCanonicalizer()
        mol = Chem.MolFromSmiles(smi)
        mol1 = tc.canonicalize(mol)
        smi1 = Chem.MolToSmiles(mol1)
        return smi1


stantaut = StandardizeTautomers()
stantaut.standardize('delaney-processed.csv', 'out-delaney-processed.csv')
stantaut.standardize('Lipophilicity.csv', 'out-Lipophilicity.csv')

SMILES_COLUMN = 'standardized_smiles' # название предобработанных данных
VALUE_COLUMN = 'exp'

DATASET_INPUT_PATH = 'out-delaney-processed.csv' #название предоработанного файла
DATASET_INPUT_PATH_lip = 'out-Lipophilicity.csv' #название предоработанного файла

data_reader = DatasetsHolder()#класс, читающий данные
data = data_reader.read_datasets(DATASET_INPUT_PATH)# метод
data_lip = data_reader.read_datasets(DATASET_INPUT_PATH_lip)# метод
data.drop_duplicates()
data

,Unnamed: 0,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles,standardized_smiles
0,0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.770,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,N#CC(OC1OC(COC2OC(CO)C(O)C(O)C2O)C(O)C(O)C1O)c...
1,1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.300,Cc1occc1C(=O)Nc2ccccc2,Cc1occc1C(=O)Nc1ccccc1
2,2,citral,-2.579,1,152.237,0,0,4,17.07,-2.060,CC(C)=CCCC(C)=CC(=O),CC(C)=CCCC(C)=CC=O
3,3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.870,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21
4,4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.330,c1ccsc1,c1ccsc1
...,...,...,...,...,...,...,...,...,...,...,...,...
1123,1123,halothane,-2.608,1,197.381,0,0,0,0.00,-1.710,FC(F)(F)C(Cl)Br,FC(F)(F)C(Cl)Br
1124,1124,Oxamyl,-0.908,1,219.266,1,0,1,71.00,0.106,CNC(=O)ON=C(SC)C(=O)N(C)C,CNC(=O)ON=C(SC)C(=O)N(C)C
1125,1125,Thiometon,-3.323,1,246.359,0,0,7,18.46,-3.091,CCSCCSP(=S)(OC)OC,CCSCCSP(=S)(OC)OC
1126,1126,2-Methylbutane,-2.245,1,72.151,0,0,1,0.00,-3.180,CCC(C)C,CCC(C)C


In [5]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [10]:
data.to_csv('/content/drive/MyDrive/standData/data.csv')

FileNotFoundError: ignored

In [12]:
class data_preprocessor:
  def __init__(self):
    self.symbols = []
    self.seq_length = -1
    self.seq_lengths = []


  def preprocess_data(self, df, target):
    self.tokenization(df)
    self.create_vocab()
    X = []
    for x in df['tokens']:
        X.append(self.pad_input(self.seq2ind(x.split())))
    y = df[target]
    return X, y


  def tokenization(self, df):
    """
    Tokenize all SMILES in df dataframe
    """
    df['tokens'] = df['standardized_smiles'].apply(self.smiles_tokenization)
    self.seq_length = max(self.seq_lengths)
    return df

  def smiles_tokenization(self, smi):
    """
    Tokenize a SMILES molecule or reaction
    """
    smi.strip()
    # https://docs.python.org/3/library/re.html
    import re
    pattern = "[0-9]|Cl?|N|I|Se?|O|H|F|Br?|P|cl?|n|i|se?|s|o|h|br?|p|\(|\)|\=|\[|\]|@|#|-|\+|\\\\|\.|%|\/"
    regex = re.compile(pattern)
    
    tokens = regex.findall(smi)
    self.seq_lengths.append(len(tokens))
    self.symbols += tokens
    self.symbols = list(set(self.symbols))
    try:
        assert smi == ''.join(tokens)
    except:
        print(smi)
        print(''.join(tokens))
    return ' '.join(tokens)

  def create_vocab(self):
    """
    Create vocabulary of the symbols
    """
    self.token2idx = {o:i for i,o in enumerate(self.symbols)}
    self.idx2token = {i:o for i,o in enumerate(self.symbols)}

  def seq2ind(self, seq):
    return [self.token2idx[x] for x in seq]

  def pad_input(self, seq):
    features = np.zeros((self.seq_length),dtype=int)
    features[-len(seq):] = np.array(seq)[:self.seq_length]
    return features
  

In [13]:
dst = data_preprocessor()
X, y = dst.preprocess_data(data, 'measured log solubility in mols per litre')
X_lip, y_lip = dst.preprocess_data(data_lip, 'exp')

# Делим данные на тренировочную и тестовую выборки

In [14]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train_lip, X_test_lip, y_train_lip, y_test_lip = train_test_split(X_lip, y_lip)

# Создаем загрузчики данных для Pytorch


  * Преобразуйте данные в numpy array
  * сделайте torch.tensor при помощи torch.from_numpy
  * создайте датасет torch.utils.data.TensorDataset
  * сделайте из него DataLoader


In [18]:
import torch
from sklearn.metrics import r2_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

X_train = torch.tensor(np.array(X_train))
y_train = torch.tensor(np.array(y_train))
X_test = torch.tensor(np.array(X_test))
y_test = torch.tensor(np.array(y_test))

X_train_lip = torch.tensor(np.array(X_train_lip))
y_train_lip = torch.tensor(np.array(y_train_lip))
X_test_lip = torch.tensor(np.array(X_test_lip))
y_test_lip = torch.tensor(np.array(y_test_lip))


trainset = torch.utils.data.TensorDataset(X_train, y_train) 
testset = torch.utils.data.TensorDataset(X_test, y_test) 
batch_size = 100
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True) 

trainset_lip = torch.utils.data.TensorDataset(X_train_lip, y_train_lip) 
testset_lip = torch.utils.data.TensorDataset(X_test_lip, y_test_lip) 
batch_size = 100
trainloader_lip = torch.utils.data.DataLoader(trainset_lip, batch_size=batch_size, shuffle=True)
testloader_lip = torch.utils.data.DataLoader(testset_lip, batch_size=batch_size, shuffle=True) 


# Построение модели

## Сверточная нейросеть

In [11]:
class  CharCNN(nn.Module):
    def __init__(self, seq_len):
        super(CharCNN, self).__init__()
        self.embedding = nn.Embedding(len(dst.token2idx) + 1, 100)

        self.conv1 = nn.Sequential(
            nn.Conv1d(seq_len, 128, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(3968, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1)
        )
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.conv1(x)


        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

model = CharCNN(dst.seq_length)

In [12]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


**TO-DO**
1. Создайте optimizer, criterion
2. Переместите модель на GPU
3. Добавьте процедуру обучения

In [13]:
model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 5

# надо выбрать количество эпох, чтобы останавливалось, когда loss перестает менятся на валидационной выборке
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:  # вот и пригодился DataLoader - удобно и быстро поставляет батчи
        images.to(device)
        labels.to(device)
        # !!! Обнулить градиенты перед каждой итерацией !!!
        # Иначе градиенты будут накапливаться и работать будет неправильно
        optimizer.zero_grad()
        
        # Текущее предсказание модели
        output = model(images)
   
        # Лосс функция: насколько плохо сейчас предсказывает модель, сравнивая с настоящими классами?
        loss = criterion(output.float(), labels.float())
        
        # Здесь берутся градиенты лосса по весам сети с помощью backpropogation - правила сложной производной (chain rule)
        loss.backward()
        
        # Здесь оптимизитор делает шаг весов нейронной сети в сторону, обратную градиенту
        optimizer.step()
        
        running_loss += loss.item()

    # TODO: добавить валидацию. После каждой эпохи обучения считать средний loss на всей валидационной выборке и выводить вместе
    # с training loss. Это нужно, чтобы контролировать, не переобучается ли нейросеть.
    # Надо использовать обертку with torch.no_grad(): тк градиенты на брать не нужно будет.
    val_labels = []
    val_preds = []
    test_loss = 0
    with torch.no_grad():
        labels = torch.Tensor([]).to(device)
        preds = torch.Tensor([]).to(device)
        for x, y in testloader:
            y = y.to(device)
            labels = torch.cat((labels, y)) 
            x = x.to(device)
            output = model(x)
            preds = torch.cat((preds, output))
        loss = criterion(preds, labels.long())
    # TODO в print добавить лосс на тестовых данных
        #_, predictions = torch.max(torch.as_tensor(preds).data, 1)
        #acc = (labels==predictions).sum()/len(labels)
        r2_sc = r2_score(labels, preds)
    print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(trainloader)))
    print("Epoch {} - R2 score: {}".format(epoch, r2_sc))
    print("Epoch {} - Test loss: {}".format(epoch, loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([400])) that is different to the input size (torch.Size([400, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1050])) that is different to the input size (torch.Size([1050, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(i

Epoch 0 - Training loss: 8.414391785860062
Epoch 0 - R2 score: -0.2904103936754283
Epoch 0 - Test loss: 1.98223078250885


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([400])) that is different to the input size (torch.Size([400, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1050])) that is different to the input size (torch.Size([1050, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(i

Epoch 1 - Training loss: 3.298681691288948
Epoch 1 - R2 score: -0.2954511185866746
Epoch 1 - Test loss: 1.9992791414260864


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([400])) that is different to the input size (torch.Size([400, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1050])) that is different to the input size (torch.Size([1050, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(i

...

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([400])) that is different to the input size (torch.Size([400, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1050])) that is different to the input size (torch.Size([1050, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(i

Epoch 3 - Training loss: 1.8391116857528687
Epoch 3 - R2 score: -0.31375982460622764
Epoch 3 - Test loss: 1.3987126350402832


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([400])) that is different to the input size (torch.Size([400, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 4 - Training loss: 1.7553962022066116
Epoch 4 - R2 score: -0.15764451084796205
Epoch 4 - Test loss: 1.5806171894073486


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1050])) that is different to the input size (torch.Size([1050, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


## RNN

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

## LSTM

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

Почитать - https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Посмотреть - https://stepik.org/course/54098/syllabus (Языковые модели и генерация текста) 

In [13]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # Сначала создаем слой, который будет тренироваться для 
        # перевода исходных последовательностей в вектора
        # nn.Embedding

        self.embedding = nn.Embedding(len(dst.token2idx) + 1, embedding_dim)

        # Добавляем LSTM слой
        # nn.LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers) 

        # Добавляем DropOut
        self.dropout = nn.Dropout(p=drop_prob)

        # Линейный слой для вывода предсказания 
        self.fc = nn.Linear(hidden_dim, 1)


        
    def forward(self, x):
        batch_size = x.size(0)
        x = x.long()


        # создаем эмбеддинг
        embeds = self.embedding(x) 
        lstm_out, hidden = self.lstm(embeds) 
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    


In [14]:
vocab_size = len(dst.token2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

In [15]:
model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

**TO-DO**
1. Создайте optimizer, criterion
2. Переместите модель на GPU
3. Добавьте процедуру обучения


In [16]:
from tqdm import tqdm

  0%|          | 0/32 [00:00<?, ?it/s]

In [17]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 50

# надо выбрать количество эпох, чтобы останавливалось, когда loss перестает менятся на валидационной выборке
for epoch in range(epochs):
    running_loss = 0
    for images, labels in tqdm(trainloader):  # вот и пригодился DataLoader - удобно и быстро поставляет батчи
        images.to(device)
        labels.to(device)
        # !!! Обнулить градиенты перед каждой итерацией !!!
        # Иначе градиенты будут накапливаться и работать будет неправильно
        optimizer.zero_grad()
        
        # Текущее предсказание модели
        output = model(images)[0]
   
        # Лосс функция: насколько плохо сейчас предсказывает модель, сравнивая с настоящими классами?
        loss = criterion(output.float(), labels[0].float())
        
        # Здесь берутся градиенты лосса по весам сети с помощью backpropogation - правила сложной производной (chain rule)
        loss.backward()
        
        # Здесь оптимизитор делает шаг весов нейронной сети в сторону, обратную градиенту
        optimizer.step()
        
        running_loss += loss.item()

    # TODO: добавить валидацию. После каждой эпохи обучения считать средний loss на всей валидационной выборке и выводить вместе
    # с training loss. Это нужно, чтобы контролировать, не переобучается ли нейросеть.
    # Надо использовать обертку with torch.no_grad(): тк градиенты на брать не нужно будет.
    val_labels = []
    val_preds = []
    test_loss = 0
    with torch.no_grad():
        labels = torch.Tensor([]).to(device)
        preds = torch.Tensor([]).to(device)
        for x, y in testloader:
            y = y.to(device)
            labels = torch.cat((labels, y)) 
            x = x.to(device)
            output = model(x)[0]
            preds = torch.cat((preds, output))
        loss = criterion(preds, labels.long())
    # TODO в print добавить лосс на тестовых данных
        #_, predictions = torch.max(torch.as_tensor(preds).data, 1)
        #acc = (labels==predictions).sum()/len(labels)
        r2_sc = r2_score(labels, preds)
    print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(trainloader)))
    print("Epoch {} - R2 score: {}".format(epoch, r2_sc))
    print("Epoch {} - Test loss: {}".format(epoch, loss))


  0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

 97%|█████████▋| 31/32 [06:30<00:12, 12.45s/it]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

100%|██████████| 32/32 [06:36<00:00, 12.40s/it]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 0 - Training loss: 4.362435053277295
Epoch 0 - R2 score: -0.016617005425981546
Epoch 0 - Test loss: 1.541142463684082


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

 97%|█████████▋| 31/32 [06:34<00:12, 12.69s/it]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

100%|██████████| 32/32 [06:40<00:00, 12.52s/it]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1 - Training loss: 1.758123167208396
Epoch 1 - R2 score: -0.37829142868215215
Epoch 1 - Test loss: 1.4035755395889282


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

 97%|█████████▋| 31/32 [06:32<00:12, 12.68s/it]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

100%|██████████| 32/32 [06:38<00:00, 12.46s/it]

  0%|          | 0/32 [00:00<?, ?it/s]

...

Epoch 47 - Training loss: 1.204819868085906
Epoch 47 - R2 score: -0.02064932499889638
Epoch 47 - Test loss: 1.5393133163452148


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

 97%|█████████▋| 31/32 [06:40<00:12, 12.91s/it]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

100%|██████████| 32/32 [06:46<00:00, 12.70s/it]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 48 - Training loss: 1.414389488985762
Epoch 48 - R2 score: -0.016314395967680584
Epoch 48 - Test loss: 1.470029592514038


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

 97%|█████████▋| 31/32 [06:39<00:12, 12.92s/it]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

100%|██████████| 32/32 [06:46<00:00, 12.69s/it]


Epoch 49 - Training loss: 2.045391605759505
Epoch 49 - R2 score: -0.151425606185839
Epoch 49 - Test loss: 1.3107956647872925
